In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import joblib
import pickle
from mplsoccer.pitch import Pitch
from ipynb.fs.defs.PassesData import get_df_passes
from ipynb.fs.defs.ExpectedGoalsData import get_df_shots
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [2]:
plt.style.use('fivethirtyeight')
params = {'axes.titlesize': 'x-large',
          'axes.labelsize': 'large', 
          'xtick.labelsize':'large',
          'ytick.labelsize':'large'}
plt.rcParams.update(params)

In [3]:
matches = pd.read_excel('extracted_data/matches.xlsx')
players = pd.read_excel('extracted_data/players.xlsx')

In [4]:
def get_player_name(player_id):
    if pd.notna(players.loc[players['player_id'] == player_id, 'player_nickname'].iloc[0]):
        return players.loc[players['player_id'] == player_id, 'player_nickname'].iloc[0]
    else:
        return players.loc[players['player_id'] == player_id, 'player_name'].iloc[0]

In [5]:
def get_competition_data(comp_matches):
    # get all events from competition matches
    events_folder = 'data/events'
    events = pd.concat([pd.DataFrame(json.load(open(os.path.join(events_folder, str(match_id) + '.json'), encoding='utf8'))).assign(match_id=match_id) for match_id in comp_matches['match_id'].unique()])
    events.reset_index(inplace=True)
    # add predictions for the pass accuracy
    comp_passes = get_df_passes(events)
    passes_model = joblib.load('models/pass_accuracy_model.pkl')
    with open("models/passes_selected_features.txt", "rb") as fp:
         passes_selected_features = pickle.load(fp)
    comp_passes.loc[:, 'pred_accuracy'] = passes_model.predict_proba(comp_passes[passes_selected_features])[:, 1]
    # add expected goals predictions
    comp_shots = get_df_shots(events)
    xg_model = joblib.load('models/expected_goals_model.pkl')
    with open("models/xg_selected_features.txt", "rb") as fp:
         xg_selected_features = pickle.load(fp)
    comp_shots.loc[:, 'pred_xg'] = xg_model.predict_proba(comp_shots[xg_selected_features])[:, 1]
    # add the ids to the passes and shots
    events.loc[:, 'pass_event'] = [1 if events['type'].iloc[i]['id'] == 30 else 0 for i in range(len(events))]
    events.loc[:, 'shot_event'] = [1 if events['type'].iloc[i]['id'] == 16 else 0 for i in range(len(events))]
    comp_passes.loc[:, 'id'] = events.loc[events['pass_event'] == 1, 'id'].reset_index(drop=True)
    comp_shots.loc[:, 'id'] = events.loc[events['shot_event'] == 1, 'id'].reset_index(drop=True)
    events = events.loc[(events['pass_event'] == 1) | (events['shot_event'] == 1), ['id', 'match_id', 'possession', 'pass_event', 'shot_event']]
    return events, comp_passes, comp_shots

In [6]:
# calculate the playtime for each player as a dict with the player_id as the key and the minutes played as the value
def calculate_players_playtime(comp_matches):
    # get all events from competition matches
    events_folder = 'data/events'
    events = pd.concat([pd.DataFrame(json.load(open(os.path.join(events_folder, str(match_id) + '.json'), encoding='utf8'))).assign(match_id=match_id) for match_id in comp_matches['match_id'].unique()])
    events.reset_index(inplace=True)
    # calculate the playtime for each player
    players_playtime = {}
    for match_id in comp_matches['match_id'].unique():
        total_minutes = events.loc[events['match_id'] == match_id, 'minute'].max()
        for _, tactic in events.loc[(events['match_id'] == match_id) & (~events['tactics'].isna()), 'tactics'].iloc[:2].iteritems():
            for player in pd.DataFrame(tactic['lineup'])['player']:
                if player['id'] in players_playtime:
                    players_playtime[player['id']] += total_minutes
                else:
                    players_playtime[player['id']] = total_minutes
        for _, sub in events.loc[(events['match_id'] == match_id) & (~events['substitution'].isna()), ['substitution', 'player', 'minute']].iterrows():
            on_player_id = sub['substitution']['replacement']['id']
            off_player_id = sub['player']['id']
            remaining_minutes = (total_minutes - sub['minute'])
            if on_player_id in players_playtime:
                players_playtime[on_player_id] += remaining_minutes
            else:
                players_playtime[on_player_id] = remaining_minutes
            players_playtime[off_player_id] -= remaining_minutes
    return players_playtime

In [7]:
# add the xg-value of a shot to all passes that belong to the chain of passes that led to the shot
def calculate_xg_chain(comp_events, comp_passes, comp_shots):
    comp_passes.loc[:, 'xg_chain'] = 0
    for match_id in comp_events['match_id'].unique():
        match_events = comp_events[comp_events['match_id'] == match_id]
        for possession in match_events['possession'].unique():
            shot_events = match_events[(match_events['possession'] == possession) & (match_events['shot_event'] == 1)]
            if len(shot_events) > 0:
                shot_xg = comp_shots.loc[comp_shots['id'] == shot_events['id'].iloc[0], 'pred_xg'].iloc[0]
                pass_chain_ids = match_events.loc[(match_events['possession'] == possession) & (match_events['pass_event'] == 1), 'id']
                comp_passes.loc[comp_passes['id'].isin(pass_chain_ids), 'xg_chain'] = shot_xg
    return comp_passes

In [8]:
# get the xg-chain-value per match for every player
def get_xg_chain(comp_passes, comp_shots, players_playtime):
    df_xg_chain = pd.DataFrame(columns=['player_id', 'Position', 'Spieler', 'Anzahl Spiele', 'Spielminuten', 'xG-Chain', 'xG-Chain/90',
                                        'pre-xG-Chain', 'pre-xG-Chain/90'])
    for player_id in comp_passes['player_id'].unique():
        player_passes = comp_passes[comp_passes['player_id'] == player_id]
        player_shots = comp_shots[comp_shots['player_id'] == player_id]
        num_matches = len(player_passes['match_id'].unique())
        num_minutes = players_playtime[player_id]
        xg_chain_sum = np.round(np.sum(player_passes['xg_chain'].unique()) + player_shots['pred_xg'].sum(), 2)
        pre_xg_chain_sum = np.round(np.sum(player_passes.loc[(player_passes['shot_assist'] == 0) & (player_passes['goal_assist'] == 0), 'xg_chain'].unique()), 2)
        pos = player_passes['position'].iloc[0]
        position = 'Torwart' if pos == 1 else 'Verteidiger' if pos <= 8 else 'Mittelfeldspieler' if pos <= 21 else 'Stürmer'
        df_xg_chain = df_xg_chain.append(pd.Series({'player_id': player_id, 'Position': position, 'Spieler': get_player_name(player_id),
                                                    'Anzahl Spiele': num_matches, 'Spielminuten': num_minutes, 'xG-Chain': xg_chain_sum, 
                                                    'xG-Chain/90': np.round(90 * xg_chain_sum / num_minutes, 2), 'pre-xG-Chain': pre_xg_chain_sum, 
                                                    'pre-xG-Chain/90': np.round(90 * pre_xg_chain_sum / num_minutes, 2)}), ignore_index=True)
    return df_xg_chain

In [9]:
# calculate the effectiveness for a pass: -2 für Fehlpässe, die zu einem Torschuss des Gegners führen; 
# -1 für harmlose Fehlpässe; 0 für neutrale Pässe; 1 für raumgewinnende Pässe; 2 für Pässe, die zu einem Torschuss führen
def calculate_pass_effectiveness(the_pass, comp_events):
    if the_pass['complete'] == 0:
        event = comp_events[comp_events['id'] == the_pass['id']].iloc[0]
        next_possession_chain = comp_events[(comp_events['match_id'] == event['match_id']) & (comp_events['possession'] == event['possession']+1)]
        if (next_possession_chain['pass_event'].sum() <= 4) & (next_possession_chain['shot_event'].sum() > 0):
            return -2
        else:
            return -1
    else:
        if (the_pass['shot_assist'] == 1) | (the_pass['goal_assist'] == 1):
            return 2
        elif ((the_pass['location_x'] > 60) & (the_pass['end_location_x'] > the_pass['location_x'])) | (the_pass['end_location_x'] - 10 > the_pass['location_x']) | (the_pass['switch'] == 1) | (the_pass['cross'] == 1) | (the_pass['cut_back'] == 1):
            return 1
        else:
            return 0

In [10]:
# add a score to every pass for the effectiveness of a pass
def calculate_pass_effectivenesses(comp_passes, comp_events):
    comp_passes.loc[:, 'pass_effectiveness'] = [calculate_pass_effectiveness(row, comp_events) for _, row in comp_passes.iterrows()]
    return comp_passes

In [11]:
# add a score to every pass that combines the effectiveness and the difficulty of a pass
def calculate_pass_scores(comp_passes):
    comp_passes.loc[:, 'pass_score'] = [row['pass_effectiveness']*(1-row['pred_accuracy']) if row['pass_effectiveness'] >= 0 else row['pass_effectiveness']*row['pred_accuracy'] for _, row in comp_passes.iterrows()]
    return comp_passes

In [12]:
# get the sum of passing scores per match for every player
def get_pass_scores(comp_passes, players_playtime):
    df_pass_scores = pd.DataFrame(columns=['player_id', 'Position', 'Spieler', 'Anzahl Spiele', 'Spielminuten', 'Passeffekt', 
                                           'Passeffekt/90', 'Pässe', '-2', '-1', '0', '1', '2', 'Passwert', 'Passwert/90'])
    for player_id in comp_passes['player_id'].unique():
        player_passes = comp_passes[comp_passes['player_id'] == player_id]
        num_matches = len(player_passes['match_id'].unique())
        num_minutes = players_playtime[player_id]
        pass_effect_sum = player_passes['pass_effectiveness'].sum()
        pass_score_sum = np.round(player_passes['pass_score'].sum(), 2)
        pos = player_passes['position'].iloc[0]
        position = 'Torwart' if pos == 1 else 'Verteidiger' if pos <= 8 else 'Mittelfeldspieler' if pos <= 21 else 'Stürmer'
        df_pass_scores = df_pass_scores.append(pd.Series({'player_id': player_id, 'Position': position, 'Spieler': get_player_name(player_id),
                                                          'Anzahl Spiele': num_matches, 'Spielminuten': num_minutes,
                                                          'Passeffekt': pass_effect_sum, 
                                                          'Passeffekt/90': np.round(90 * pass_effect_sum / num_minutes, 2), 
                                                          'Pässe': len(player_passes), 
                                                          '-2': len(player_passes[player_passes['pass_effectiveness'] == -2]), 
                                                          '-1': len(player_passes[player_passes['pass_effectiveness'] == -1]),
                                                          '0': len(player_passes[player_passes['pass_effectiveness'] == 0]), 
                                                          '1': len(player_passes[player_passes['pass_effectiveness'] == 1]), 
                                                          '2': len(player_passes[player_passes['pass_effectiveness'] == 2]),
                                                          'Passwert': pass_score_sum, 
                                                          'Passwert/90': np.round(90 * pass_score_sum / num_minutes, 2)}), ignore_index=True)
    return df_pass_scores

Calculation of Expected Goals Chain Values for the World Cup 2018

In [ ]:
# get data for the world cup 2018
comp_id = 43
comp_matches = matches[matches['competition_id'] == comp_id]
wc_events, wc_passes, wc_shots = get_competition_data(comp_matches)

In [ ]:
wc_players_playtime = calculate_players_playtime(comp_matches)

In [ ]:
wc_passes = calculate_xg_chain(wc_events, wc_passes, wc_shots)

In [ ]:
df_xg_chain = get_xg_chain(wc_passes, wc_shots, wc_players_playtime)

In [ ]:
df_xg_chain[df_xg_chain['Spielminuten'] >= 200].sort_values('xG-Chain/90', ascending=False).head(20)

In [ ]:
wc_passes = calculate_pass_effectivenesses(wc_passes, wc_events)

In [ ]:
wc_passes = calculate_pass_scores(wc_passes)

In [ ]:
df_pass_scores = get_pass_scores(wc_passes, wc_players_playtime)

In [ ]:
df_pass_scores[(df_pass_scores['Spielminuten'] >= 200) & (df_pass_scores['Position'] == 'Torwart')].sort_values('Passwert/90', ascending=False).head(10)

In [ ]:
df_pass_scores[(df_pass_scores['Spielminuten'] >= 200) & (df_pass_scores['Position'] == 'Verteidiger')].sort_values('Passwert/90', ascending=False).head(10)

In [ ]:
df_pass_scores[(df_pass_scores['Spielminuten'] >= 200) & (df_pass_scores['Position'] == 'Mittelfeldspieler')].sort_values('Passwert/90', ascending=False).head(20)

In [ ]:
df_pass_scores.loc[(df_pass_scores['Spielminuten'] >= 200) & (df_pass_scores['Position'] == 'Mittelfeldspieler'), ['Spieler', 'Anzahl Spiele', 'Spielminuten', 'Passeffekt', 'Passeffekt/90', 'Pässe', '-2', '-1', '0', '1', '2']].sort_values('Passeffekt/90', ascending=False).head(20)

In [ ]:
df_pass_scores[(df_pass_scores['Spielminuten'] >= 200) & (df_pass_scores['Position'] == 'Stürmer')].sort_values('Passwert/90', ascending=False).head(10)